In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Sep 14 16:32:25 2019

@author: adity
"""

import mysql.connector
from faker import Faker
import random
import time
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import numpy as np


#Global Constants
operation_count = 0
below_zero = []
user_count = 0
product_count = 0


def setSQLCredentials():
    # Setting SQL Credentials 
    # These credentials MUST be changed and set as required in order to run the below program
    global user
    global pword
    global host
    global database
    user='root'
    pword='password'
    host='localhost'
    database='csci729_transaction'

def setUserCount():
    global user
    global pword
    global host
    global database
    global product_count
    # Setting product names as Product1, Product2, and so on. This query is for getting the product count to avoid duplicate product entry
    cnx = mysql.connector.connect(user=user, password=pword,
                              host=host,
                              database=database)
    mycursor = cnx.cursor()
    mycursor.execute("SELECT count(*) FROM product")
    myresult = mycursor.fetchall()
    product_count = myresult[0][0]
    cnx.close()

def setProductCount():
    global user
    global pword
    global host
    global database
    global user_count
    # Setting product names as user1, user2, and so on. This query is for getting the user count to avoid duplicate user entry
    cnx = mysql.connector.connect(user=user, password=pword,
                              host=host,
                              database=database)
    mycursor = cnx.cursor()
    mycursor.execute("SELECT count(*) FROM users")
    myresult = mycursor.fetchall()
    user_count = myresult[0][0]
    cnx.close()


# Methods to generate parameters for the implementing the application operations

def createAccount_genParameter():
    global user_count
    fake = Faker()
    firstName, lastName = fake.first_name(),fake.last_name()
    username = 'user'+str(user_count)
    password = 'password' + str(user_count)
    user_count += 1
    return username, password, firstName, lastName
            
def AddProduct_genParameter():
    global product_count
    fake = Faker()
    name = 'product' + str(product_count)
    desc = fake.bs()[:44]
    price = fake.random_int(1, 999)
    quantity = fake.random_int(100, 100000)
    product_count += 1
    return name, desc, price, quantity    
        
def PostReview_genParameter():
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    rand = str(random.randint(0,1000))
    username = 'user' + rand
    password = 'password' + rand
    mycursor.execute("SELECT productID FROM product ORDER BY RAND() LIMIT 1")
    myresult = mycursor.fetchall()
    productID = myresult[0][0]
    #productID = random.randint(0,10000)
    fake = Faker()
    rating = random.randint(4,10)
    reviewText = fake.catch_phrase()[:44]
    cnx.close()
    return username, password, productID, rating, reviewText

def GetAverageUserRating_genParameter():
    global user_count
    rand = str(random.randint(0,user_count))
    username = 'user' + rand
    return username

def submitOrder_genParameter():
    global user
    global pword
    global host
    global database
    global user_count
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    rand = str(random.randint(0,user_count))
    username = 'user' + rand
    password = 'password' + rand
    mycursor.execute("SELECT productID FROM product ORDER BY RAND() LIMIT 10")
    myresult = mycursor.fetchall()
    print(myresult)
    productID = myresult
    listOfProductsAndQuantities = []
    for i in range(10):
        rand = str(random.randint(10,100))
        listOfProductsAndQuantities.append([productID[i][0], rand])
    cnx.close()
    return username, password, listOfProductsAndQuantities

def UpdateStockLevel_genParameter():
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    mycursor.execute("SELECT productID FROM product ORDER BY RAND() LIMIT 1")
    myresult = mycursor.fetchall()
    productID = myresult[0][0]
    itemCountToAdd = random.randint(1000,100000)
    cnx.close()
    return productID, itemCountToAdd    

def GetProductAndReviews_genParameter():
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    mycursor.execute("SELECT productID FROM product ORDER BY RAND() LIMIT 1")
    myresult = mycursor.fetchall()
    productID = myresult[0][0]
    cnx.close()
    return productID


# Implementing Application operations 

def createAccount(username, password, firstName, lastName):
    #. Establishes a new account for the user. This should fail if the username already exists. 
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    cnx.start_transaction(isolation_level='SERIALIZABLE')
    try:
        args = [username, password, firstName, lastName]
        mycursor.callproc('CreateAccount', args)
        cnx.commit()
        print("AddProduct Success!!")
    except:
        print("AddProduct Failed")
        cnx.rollback()
    cnx.close()

def AddProduct(name, desc, price, quantity):
    # A new product is added to the database with the given name and description and the given initial stock value. This operation should provide an ID for the product which can be used in future operations. 
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    cnx.start_transaction(isolation_level='SERIALIZABLE')
    try:
        args = [name, desc, str(price), quantity]
        mycursor.callproc('AddProduct', args)
        cnx.commit()
        print("AddProduct Success!!")
    except:
        print("AddProduct Failed")
        cnx.rollback()
    cnx.close()

def GetAverageUserRating(username):
    #Get the average rating value for a given user by adding the ratings for all products and dividing by the total number of reviews the user has provided. 
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    cnx.start_transaction(consistent_snapshot=bool, isolation_level='READ COMMITTED')
    try:
        args = [username, 0]
        rating = mycursor.callproc('GetAverageUserRating', args)[1]
        print(rating)
        cnx.commit()
        print("GetAverageUserRating Success!!")
    except:
        print("GetAverageUserRating Failed")
        cnx.rollback()
    cnx.close()
 

#userName,password,listOfProductsAndQuantities    
arr = ['user1','password1',[[12,34],[33,12]]]

def submitOrder(username, password, listOfProductsAndQuantities):
    #SubmitOrder(username, password, listOfProductsAndQuantities)  
    global user
    global pword
    global host
    global database
    global user_count
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    cnx.start_transaction(isolation_level='SERIALIZABLE')
    try:
        arg1 = [username,password,0]
        print(arg1)
        userID = mycursor.callproc('AUTHORISATION', arg1)[2]
        print(userID)
        #PROCEDURE `check_product_quantity`(in _productID int, in _qty int, out is_true boolean)
        quantityCheck = True
        for elem in listOfProductsAndQuantities:
            arg2 = [elem[0],elem[1],False]
            quantityCheck &= mycursor.callproc('check_product_quantity', arg2)[2]
        #print(quantityCheck)
        if quantityCheck == 1:
            arg3 = [userID, 0]
            OrderID = mycursor.callproc('SUBMIT_ORDER', arg3)[1]
            print(OrderID)
            for elem in listOfProductsAndQuantities:
                arg4 = [OrderID, elem[0], elem[1]]
                mycursor.callproc('insert_order_item', arg4)
            cnx.commit()
        else:
            cnx.rollback()
            print('submitOrder Success!!')
    except:
        print('submitOrder failed')
        cnx.rollback()
    cnx.close()
    
def UpdateStockLevel(productID, itemCountToAdd):
    #Adds new inventory associated with the product. The given number of items should be added to the current stock level. 
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    cnx.start_transaction(isolation_level='SERIALIZABLE')
    try:
        args = [productID, itemCountToAdd]
        mycursor.callproc('UpdateStockLevel', args)
        cnx.commit()
        print("UpdateStockLevel Success!!")
    except:
        print("UpdateStockLevel Failed")
        cnx.rollback()
    cnx.close()


def PostReview(username, password, productID, rating, reviewText):
    # First authorizes the user (as above) and then submits a review. Each user should also only be able to post a single review for a given product. 
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    cnx.start_transaction(consistent_snapshot=bool, isolation_level='SERIALIZABLE')
    try:
        args = [username, password, productID, rating, reviewText]
        mycursor.callproc('PostReview', args)
        cnx.commit()
        print("PostReview Sucess!!")
    except:
        print("PostReview Failed")
        cnx.rollback()
    cnx.close()

def GetProductAndReviews(productID):
    # Return the product information and all the reviews for the given product including the username of the reviewing user, the rating, and the text of the review. 
    global user
    global pword
    global host
    global database
    cnx = mysql.connector.connect(user=user, password=pword,
                          host=host,
                          database=database)
    mycursor = cnx.cursor()
    cnx.start_transaction(consistent_snapshot=bool, isolation_level='READ COMMITTED')
    try:
        #args = [productID,'','',0]
        #output = mycursor.callproc('GetProductAndReviews', args)
        #print(output)
        query = 'SELECT users.username, review.reviewText, review.rating from review INNER JOIN users on users.userID = review.userID where productId =' +str(productID) + ';'
        mycursor.execute(query)
        myresult = mycursor.fetchall()
        print(f'productID\t{productID}')
        for username,reviewText,rating in myresult:
            print(f'{username}\t{reviewText}\t{rating}')
        cnx.commit()
        print("GetProductAndReviews Sucess!!")
    except:
        print("GetProductAndReviews Failed")
        cnx.rollback()
    cnx.close()
 

def initialise():
    # Intitialise the database as per given requirements.
    #1. Run the AddProduct operation 10,000 times to add random new products. 
    for i in range(10000):
        name, desc, price, quantity = AddProduct_genParameter()
        AddProduct(name, desc, price, quantity)
    
    #2. Run the CreateAccount operation 1,000 times to create random new users. 
    for i in range(1000):
        username, password, firstName, lastName = createAccount_genParameter()
        createAccount(username, password, firstName, lastName)
        
    #3. Run the PostReview operation 20,000 times to create random reviews of products by random users. 
    for i in range(20000):
        username, password, productID, rating, reviewText = PostReview_genParameter()
        PostReview(username, password, productID, rating, reviewText)
    
    #4. Run the SubmitOrder operation 10,000 times with a random user and 10 random products each time along with a random date in the past year. 
    for i in range(10000):
        username, password, listOfProductsAndQuantities = submitOrder_genParameter()
        submitOrder(username, password, listOfProductsAndQuantities)


# Helper functions to call the Application operations 
def CreateAccount_helper():
    username, password, firstName, lastName = createAccount_genParameter()
    createAccount(username, password, firstName, lastName)
    
def AddProduct_helper():
    print('Hello')
    name, desc, price, quantity = AddProduct_genParameter()
    AddProduct(name, desc, price, quantity)
    
def UpdateStockLevel_helper():
    productID, itemCountToAdd = UpdateStockLevel_genParameter() 
    UpdateStockLevel(productID, itemCountToAdd)
  
def GetProductAndReviews_helper():
    print('Hello')
    productID = GetProductAndReviews_genParameter()
    GetProductAndReviews(productID)

def GetAverageUserRating_helper():
    username = GetAverageUserRating_genParameter()
    GetAverageUserRating(username)    
    
def SubmitOrder_helper():
    username, password, listOfProductsAndQuantities = submitOrder_genParameter()
    submitOrder(username, password, listOfProductsAndQuantities)
    
def PostReview_helper():
    print('Hello')
    username, password, productID, rating, reviewText = PostReview_genParameter()
    PostReview(username, password, productID, rating, reviewText)

# Testing the performance
def test():
    global operation_count
    global below_zero
#    probability_map = {3:CreateAccount_helper, 5:AddProduct_helper, 15:UpdateStockLevel_helper, 80:GetProductAndReviews_helper, 85:GetAverageUserRating_helper, 95: SubmitOrder_helper, 100:PostReview_helper}
    curr = 0
    #Setting the time to 5 minutes
    timeout = time.time() + 60*5
    while time.time() < timeout:
     #   print(i)
         # Using thread pool executer that generates 10 thread for testing the performance
        with ThreadPoolExecutor(max_workers=10) as e:
            #Randomly generating numbers between 1 to 100 to implement the operations with given probability
            curr = random.randint(1,100)
            #curr = min(filter(lambda i: i > k, probability_map.keys()))
            if curr <= 3:
                e.submit(CreateAccount_helper)
            elif curr <= 5:
                e.submit(AddProduct_helper)
            elif curr <= 15:
                e.submit(UpdateStockLevel_helper)
            elif curr <= 80:
                e.submit(GetProductAndReviews_helper)
            elif curr <= 85:
                e.submit(GetAverageUserRating_helper)
            elif curr <= 95:
                e.submit(SubmitOrder_helper)
            elif curr <= 100:
                e.submit(PostReview_helper)
            cnx = mysql.connector.connect(user=user, password=pword,
                              host=host,
                              database=database)
            mycursor = cnx.cursor()
            mycursor.execute("SELECT count(*) FROM product where stock < 0")
            myresult = mycursor.fetchall()
            productCount = myresult[0][0]
            below_zero.append(productCount)
            cnx.close()
            operation_count += 1

def plot():
    global below_zero
    global operation_count
    plt.xlabel('Percentage of products with a stock level less than zero')
    plt.ylabel('Operations performed by all threads combined')
    plt.plot(below_zero,np.arange(operation_count))
    plt.show()
    



ModuleNotFoundError: No module named 'mysql'

SyntaxError: invalid syntax (<ipython-input-2-c67f05438eef>, line 1)

In [ ]:
def main():
    #Initialize Ennvironment
    setSQLCredentials()
    setUserCount()
    setProductCount()
    
    #Initialise Databse 
    initialise()
    
    #Test the performance by running the operations for 5 minutes
    test()
   
    #Plot the results
    plot()
    
    # The plot is a straight line at zero, before submitting every order it was
    # checked if the quantity of items available in the stock is more than the
    # count of items in the order. Hence there would not be any instances where 
    # the Stock is negative
    # Also the requiremt ask to get the quantity in percentage, but since all 
    # the values are zero, so it maps to 0 %.
    
if __name__ == '__main__':
    main()